In [186]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances

In [187]:
df_ratings = pd.read_csv('./inputs/rating_final.csv')
df_ratings.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [188]:
df_geoplaces = pd.read_csv('./inputs/geoplaces2.csv')
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rinc�n de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [189]:
df_ratings['overall_rating'] = (df_ratings['rating'] + df_ratings['food_rating'] + df_ratings['service_rating'])/3
df_ratings.head()

,userID,placeID,rating,food_rating,service_rating,overall_rating
0,U1077,135085,2,2,2,2.000000
1,U1077,135038,2,2,1,1.666667
2,U1077,132825,2,2,2,2.000000
3,U1077,135060,1,2,2,1.666667
4,U1068,135104,1,1,2,1.333333


In [190]:
df = pd.merge(df_ratings, df_geoplaces, on='placeID')
df

,userID,placeID,rating,food_rating,service_rating,overall_rating,latitude,longitude,the_geom_meter,name,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,135085,2,2,2,2.000000,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,135085,1,2,1,1.333333,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,135085,1,2,1,1.333333,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,135085,2,2,2,2.000000,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,135085,2,1,2,1.666667,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,U1061,132958,2,2,2,2.000000,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1157,U1025,132958,1,0,0,0.333333,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1158,U1097,132958,2,1,1,1.333333,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1159,U1096,132958,1,2,2,1.666667,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none


In [191]:
user_ids = df['userID'].tolist()
user_ids = list(map(lambda id : int(id[1:])-1001, user_ids))
user_ids

[76,
 107,
 80,
 55,
 133,
 115,
 131,
 119,
 35,
 83,
 112,
 103,
 105,
 44,
 15,
 6,
 12,
 0,
 134,
 31,
 61,
 91,
 108,
 121,
 97,
 32,
 88,
 26,
 28,
 85,
 136,
 45,
 89,
 58,
 48,
 1,
 76,
 52,
 125,
 124,
 75,
 21,
 115,
 23,
 131,
 119,
 18,
 112,
 8,
 6,
 98,
 53,
 100,
 31,
 61,
 108,
 121,
 97,
 32,
 88,
 76,
 54,
 52,
 80,
 0,
 55,
 133,
 115,
 23,
 131,
 35,
 70,
 48,
 103,
 44,
 8,
 134,
 61,
 108,
 84,
 97,
 2,
 90,
 28,
 136,
 113,
 60,
 56,
 24,
 58,
 5,
 1,
 76,
 52,
 63,
 124,
 21,
 115,
 23,
 3,
 70,
 44,
 8,
 15,
 98,
 53,
 31,
 134,
 77,
 97,
 85,
 127,
 72,
 12,
 67,
 66,
 102,
 122,
 62,
 27,
 86,
 67,
 59,
 122,
 20,
 22,
 49,
 27,
 86,
 67,
 102,
 30,
 81,
 27,
 86,
 67,
 66,
 102,
 81,
 27,
 49,
 86,
 42,
 67,
 66,
 102,
 81,
 27,
 42,
 66,
 102,
 106,
 122,
 62,
 49,
 66,
 102,
 106,
 59,
 122,
 22,
 79,
 81,
 27,
 67,
 66,
 81,
 49,
 86,
 102,
 69,
 81,
 122,
 27,
 42,
 102,
 122,
 62,
 86,
 106,
 79,
 62,
 86,
 67,
 43,
 29,
 71,
 116,
 19,
 50,
 43,
 117,


In [192]:
df['userID'] = pd.DataFrame(user_ids) 
df

,userID,placeID,rating,food_rating,service_rating,overall_rating,latitude,longitude,the_geom_meter,name,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,76,135085,2,2,2,2.000000,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,107,135085,1,2,1,1.333333,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,80,135085,1,2,1,1.333333,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,55,135085,2,2,2,2.000000,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,133,135085,2,1,2,1.666667,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,60,132958,2,2,2,2.000000,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1157,24,132958,1,0,0,0.333333,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1158,96,132958,2,1,1,1.333333,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1159,95,132958,1,2,2,1.666667,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none


In [193]:
place_ids = df['placeID'].tolist()
place_ids.sort()
place_ids = list(map(lambda id : (id-132560), place_ids))

new_place_ids = []
i = -1
seen = []

for place_id in place_ids:
    if place_id not in seen:
        seen.append(place_id)
        i = i+1
    new_place_ids.append(i)
    
new_place_ids

[0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 18,
 18,
 18,
 18,
 19,
 19,
 19,
 19,
 20,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 26,
 26,
 27,
 27,
 27,
 28,
 28,
 28,
 28,
 28,
 28,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 29,
 30,
 30,
 30,
 30,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,
 31,

In [194]:
df['placeID'] = pd.DataFrame(new_place_ids) 
df

,userID,placeID,rating,food_rating,service_rating,overall_rating,latitude,longitude,the_geom_meter,name,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,76,0,2,2,2,2.000000,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,107,0,1,2,1,1.333333,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,80,0,1,2,1,1.333333,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,55,0,2,2,2,2.000000,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,133,1,2,1,2,1.666667,22.150802,-100.982680,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,60,128,2,2,2,2.000000,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1157,24,129,1,0,0,0.333333,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1158,96,129,2,1,1,1.333333,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none
1159,95,129,1,2,2,1.666667,22.144979,-101.005683,0101000020957F000049095EB34A4858C15CB4BD1EE1AB...,tacos los volcanes,...,No_Alcohol_Served,none,informal,completely,low,?,quiet,t,closed,none


In [195]:
df = df[['userID', 'placeID', 'overall_rating', 'name']]
df

,userID,placeID,overall_rating,name
0,76,0,2.000000,Tortas Locas Hipocampo
1,107,0,1.333333,Tortas Locas Hipocampo
2,80,0,1.333333,Tortas Locas Hipocampo
3,55,0,2.000000,Tortas Locas Hipocampo
4,133,1,1.666667,Tortas Locas Hipocampo
...,...,...,...,...
1156,60,128,2.000000,tacos los volcanes
1157,24,129,0.333333,tacos los volcanes
1158,96,129,1.333333,tacos los volcanes
1159,95,129,1.666667,tacos los volcanes


In [196]:
n_users = df.userID.nunique()
n_items = df.placeID.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Restaurants: '+str(n_items))

Num. of Users: 138
Num of Restaurants: 130


In [197]:
train_data, test_data = train_test_split(df, test_size=0.25)

In [198]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1], line[2]] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1], line[2]] = line[3]
    
print(train_data_matrix)
print(test_data_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 1.66666667 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [199]:
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [200]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [201]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [202]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [203]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 1.4361798638807683
Item-based CF RMSE: 1.4488930694391904
